In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import torch

from tool.model import net
from tool.load import *
from tool.evaluate import *

In [ ]:
file_list = nc_list("/mnt/obsdata/isee_nlfff_v1.2/12673")

In [ ]:
def create_pair(nc_file):
    b = load_b(nc_file)

    b_norm = 2500
    input_data = b[:, :, :, 0]
    input_data = input_data[:, :, :, None] / b_norm
    output_data = b[:, :, :, :50] / b_norm

    return input_data, output_data

In [ ]:
len(file_list[::20])

20

In [ ]:
results = map(create_pair, file_list[::20])
k = list(results)

In [ ]:
k[0]

In [ ]:
k[0][1].shape, k[1][1].shape

((3, 513, 257, 50), (3, 513, 257, 50))

In [ ]:
inputs = [k[i][0] for i in range(len(k))]
outputs = [k[i][1] for i in range(len(k))]

In [ ]:
inputs[0].shape

(3, 513, 257, 1)

In [ ]:
outputs[0].shape

(3, 513, 257, 50)

In [ ]:
np.allclose(inputs[0][..., 0], outputs[0][..., 0])

True

In [ ]:
np.save('inputs.npy', inputs)
np.save('outputs.npy', outputs)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(k[0][0][2, :, :, 0].T, origin='lower')

In [ ]:
np.allclose(k[0][0][2, :, :, 0], k[0][1][2, :, :, 0])

True

In [ ]:
b = load_b(file_list[0])

In [ ]:
input_data = b[:, :, :, 0]
input_data = input_data[:, :, :, None]
input_data = torch.Tensor(input_data) / b_norm
output_data = b[:, :, :, :50]
output_data = output_data
output_data = torch.Tensor(output_data) / b_norm

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler

In [ ]:
batches_file_paths = {'inputs': 'inputs.npy', 'outputs':'outputs.npy'}

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, batches_file_paths, batch_size):
        self.batches_file_paths = batches_file_paths
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(np.load(list(self.batches_file_paths.values())[0], mmap_mode='r').shape[0] / self.batch_size).astype(np.int32)

    def __getitem__(self, idx):
        data = {k: np.copy(np.load(bf, mmap_mode='r')[idx * self.batch_size: (idx + 1) * self.batch_size])
                for k, bf in self.batches_file_paths.items()}
        return data

In [ ]:
dataset = CustomDataset(batches_file_paths, batch_size=2)
dataloaer = DataLoader(dataset, batch_size=None, num_workers=8, pin_memory=True,
                       sampler=RandomSampler(dataset, replacement=True, num_samples=10))

In [ ]:
for batch_idx, samples in enumerate(dataloaer):
    print(batch_idx)
    print(samples['inputs'].shape)
    print(samples['outputs'].shape)

0
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
1
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
2
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
3
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
4
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
5
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
6
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
7
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
8
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])
9
torch.Size([2, 3, 513, 257, 1])
torch.Size([2, 3, 513, 257, 50])


In [ ]:
from torch.optim import Adam
from torch import nn

In [ ]:
model = net().to('cuda')

In [ ]:
criterion = nn.MSELoss()

# from tool.diff import *
# def criterion(outputs, labels):
#     loss_bc = torch.mean((outputs[..., 0] - labels[..., 0])**2)

#     Bx = outputs[:, 0, ...]
#     By = outputs[:, 1, ...]
#     Bz = outputs[:, 2, ...]

#     Jx, Jy, Jz = curl(Bx, By, Bz, 1, 1, 1)
    
#     B = torch.stack([Bx, By, Bz], -1)
#     J = torch.stack([Jx, Jy, Jz], -1)

#     JxB = torch.cross(J, B, -1)
#     loss_ff = torch.linalg.norm(JxB, -1)**2

#     divB = divergence(Bx, By, Bz, 1, 1 ,1)
#     loss_div = divB**2

#     loss = loss_bc + loss_ff + loss_div

#     return loss

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
import wandb

In [ ]:
wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mgjeon. Use `wandb login --relogin` to force relogin


In [ ]:
def plots(b, B, z=0, b_norm=2500):
    fig, axs = plt.subplots(2, 3, figsize=(12, 4))

    ax = axs[0]
    ax[0].imshow(b[..., z, 0].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    ax[0].set_title('bx')
    ax[1].imshow(b[..., z, 1].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    ax[1].set_title('by')
    ax[2].imshow(b[..., z, 2].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    ax[2].set_title('bz')

    ax = axs[1]
    ax[0].imshow(B[..., z, 0].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    ax[0].set_title('Bx')
    ax[1].imshow(B[..., z, 1].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    ax[1].set_title('By')
    ax[2].imshow(B[..., z, 2].transpose(), vmin=-b_norm, vmax=b_norm, cmap='gray', origin='lower')
    ax[2].set_title('Bz')
    
    plt.tight_layout()
    plt.close()
    
    return fig

In [ ]:
wandb.watch(model, log_freq=10)

model.train()
for epoch in range(10000):
    for batch_idx, samples in enumerate(dataloaer):
        inputs = samples['inputs']
        outputs = samples['outputs']
        inputs = inputs.to('cuda')[None, ...]
        labels = outputs.to('cuda')[None, ...]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            wandb.log({"loss": loss})

            bb = outputs.cpu().detach().numpy().squeeze().transpose(1, 2, 3, 0)
            BB = labels.cpu().detach().numpy().squeeze().transpose(1, 2, 3, 0)
            fig = plots(bb, BB, b_norm=1)
            wandb.log({"img": fig})

KeyboardInterrupt: 

In [ ]:
# bb = model(input_data.to('cuda'))
bb = outputs.cpu().detach().numpy().squeeze().transpose(1, 2, 3, 0)
BB = labels.cpu().detach().numpy().squeeze().transpose(1, 2, 3, 0)

In [ ]:
fig = plots(bb, BB, b_norm=1)

In [ ]:
inputs = np.load('inputs.npy')

In [ ]:
inputs.shape

(10, 3, 513, 257, 1)